In [19]:
import pandas as pd
import numpy as np
%matplotlib inline

In [20]:
import pandas as pd
import numpy as np

def linReg(x_train, y_train, x_test):
    ones = np.ones([x_train.shape[0], 1])
    x_train = np.concatenate((ones, x_train), axis=1)
    ones = np.ones([x_test.shape[0], 1])
#  x_test = np.reshape(x_test,[x_test.shape[0],1])
    x_test = np.concatenate((ones, x_test), axis=1)
    temp_1 = np.dot(x_train.T, x_train)
    temp_1 = temp_1.astype(np.float64)
    temp_1 = np.linalg.pinv(temp_1)
    temp_2 = np.dot(x_train.T, y_train)
    theta = np.dot(temp_1, temp_2)
    preds = np.dot(x_test, theta)
    return preds

In [21]:
places =['andhra_pradesh','assam','bihar','chattissgarh','daman_diu_dadra_nagar_haveli','goa','himachal_pradesh','kerala','lakshadweep','madhya_pradesh','maharashtra','odisha','pondicherry','punjab','rajasthan','tripura','uttar_pradesh_uttarakhand','west_bengal']

In [22]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()

for place in places:
    
    df = pd.read_csv("Ground_Water_Data/ground_water_quality_in_"+place+"-2014.csv",encoding = "unicode_escape")
    df["State"] = place
   
    df1 = df1.append(df, ignore_index=True)
    if df1.empty:
        df1 = df
df = df1[df1.columns.difference(["STATION CODE","LOCATIONS","STATE"])];
df2["State"] = df["State"]
df = df.drop(["State"],1)

Now that all the datasets are combined into one, the data pd.info is used

In [23]:
type(df2)

pandas.core.frame.DataFrame

In [24]:
df.isnull().sum()

B.O.D. (mg/l) : Max : < 3 mg/l                          132
B.O.D. (mg/l) : Mean : < 3 mg/l                         132
B.O.D. (mg/l) : Min : < 3 mg/l                          132
CONDUCTIVITY (µmhos/cm) : Max                            18
CONDUCTIVITY (µmhos/cm) : Mean                           18
CONDUCTIVITY (µmhos/cm) : Min                            18
FECAL COLIFORM (MPN/100ml) : Max : < 2500 MPN/100ml     254
FECAL COLIFORM (MPN/100ml) : Mean : < 2500 MPN/100ml    254
FECAL COLIFORM (MPN/100ml) : Min : < 2500 MPN/100ml     254
NITRATE- N+ NITRITE-N (mg/l) : Max                        0
NITRATE- N+ NITRITE-N (mg/l) : Mean                       0
NITRATE- N+ NITRITE-N (mg/l) : Min                        0
TEMPERATURE ºC : Max                                     32
TEMPERATURE ºC : Mean                                    32
TEMPERATURE ºC : Min                                     32
TOTAL COLIFORM (MPN/100ml) : Max : < 5000 MPN/100ml     176
TOTAL COLIFORM (MPN/100ml) : Mean : < 50

In [36]:
df = df[df['FECAL COLIFORM (MPN/100ml) : Max : < 2500 MPN/100ml']<2500]
df = df[df['FECAL COLIFORM (MPN/100ml) : Mean : < 2500 MPN/100ml']<2500]
df = df[df['FECAL COLIFORM (MPN/100ml) : Min : < 2500 MPN/100ml']<2500]
df = df[df['TOTAL COLIFORM (MPN/100ml) : Max : < 5000 MPN/100ml']<5000]
df = df[df['TOTAL COLIFORM (MPN/100ml) : Mean : < 5000 MPN/100ml']<5000]
df = df[df['TOTAL COLIFORM (MPN/100ml) : Min : < 5000 MPN/100ml']<5000]
df = df[df['B.O.D. (mg/l) : Max : < 3 mg/l']<3]
df = df[df['B.O.D. (mg/l) : Mean : < 3 mg/l']<3]
df = df[df['B.O.D. (mg/l) : Min : < 3 mg/l']<3]

In [37]:
def mean_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(),feature + '_imp'] = np.mean(observed_values)
    
    
    #df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df


In [38]:
missing_columns = df.loc[:, df.isnull().any()].columns.values
print(missing_columns)


[]


In [39]:
def mean_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(),feature + '_imp'] = np.mean(observed_values)
    
    
    #df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df



In [40]:
missing_columns = df.loc[:, df.isnull().any()].columns.values


In [41]:
for feature in missing_columns:
    df[feature + '_imp'] = df[feature]
    df = mean_imputation(df, feature)


In [42]:

for feature in missing_columns:
    remove = np.append(missing_columns,np.array(feature+"_imp"))
    x_train = df.loc[df[feature].notnull(),df.columns.difference(remove)]
    y_train = pd.DataFrame(df.loc[df[feature].notnull(),feature])
    std = np.var(y_train)
    
    test =  pd.DataFrame(df.loc[df[feature].isnull(),df.columns.difference(remove)])
    
    xt = x_train.values
    yt = y_train.values
    tt = test.values;
    preds = np.array(linReg(xt,yt,tt))#+np.random.normal(0,std,[test.shape[0],1])
    preds = np.reshape(preds,[preds.shape[0],])
    df.loc[df[feature].isnull(),feature] = preds# + np.random.normal(0,std,[tt.shape[0],])
    
            

    

In [43]:
for feature in missing_columns:
    df.drop([feature+"_imp"],1)

In [44]:
df["STATION CODE"]= df1["STATION CODE"]
df["LOCATIONS"] = df1["LOCATIONS"]
df["State"] = df2["State"]

In [45]:
df.to_csv('Ground_water_pre_processed/filled_groundwater.csv')

In [46]:
df

,B.O.D. (mg/l) : Max : < 3 mg/l,B.O.D. (mg/l) : Mean : < 3 mg/l,B.O.D. (mg/l) : Min : < 3 mg/l,CONDUCTIVITY (µmhos/cm) : Max,CONDUCTIVITY (µmhos/cm) : Mean,CONDUCTIVITY (µmhos/cm) : Min,FECAL COLIFORM (MPN/100ml) : Max : < 2500 MPN/100ml,FECAL COLIFORM (MPN/100ml) : Mean : < 2500 MPN/100ml,FECAL COLIFORM (MPN/100ml) : Min : < 2500 MPN/100ml,NITRATE- N+ NITRITE-N (mg/l) : Max,...,pH : Min : 6.5-8.5,B.O.D. (mg/l) : Max : < 3 mg/l_imp,B.O.D. (mg/l) : Mean : < 3 mg/l_imp,B.O.D. (mg/l) : Min : < 3 mg/l_imp,CONDUCTIVITY (µmhos/cm) : Max_imp,CONDUCTIVITY (µmhos/cm) : Mean_imp,CONDUCTIVITY (µmhos/cm) : Min_imp,STATION CODE,LOCATIONS,State
4,1.000000,0.90000,0.800000,1950.0,1687.0,1520.0,0.0,0.0,0.0,8.0,...,7.2,1.000000,0.90000,0.800000,1950.0,1687.0,1520.0,1517.0,"B/W. - TUNGBHADRA RIVER NEAR KURNOOL, A.P.",andhra_pradesh
5,1.600000,1.40000,1.000000,5000.0,3720.0,2150.0,0.0,0.0,0.0,14.0,...,7.8,1.600000,1.40000,1.000000,5000.0,3720.0,2150.0,1518.0,"B/W. - NANDYAL, KURNOOL DIST., A.P.",andhra_pradesh
6,1.800000,1.40000,1.100000,1510.0,1131.0,730.0,0.0,0.0,0.0,3.1,...,7.8,1.800000,1.40000,1.100000,1510.0,1131.0,730.0,1519.0,"B/W. - NAGIRI, CHITTOOR DIST., A.P",andhra_pradesh
7,1.500000,1.30000,1.100000,1100.0,966.0,723.0,0.0,0.0,0.0,11.5,...,7.6,1.500000,1.30000,1.100000,1100.0,966.0,723.0,1520.0,"B/W. - SWARNAMUKHI RIVER, SRIKALAHASTI, CHITTO...",andhra_pradesh
8,1.000000,1.00000,1.000000,1790.0,1790.0,1790.0,9.0,9.0,9.0,5.4,...,7.7,1.000000,1.00000,1.000000,1790.0,1790.0,1790.0,1521.0,"O/W. - NEAR RAMA TEMPLE , WARD NO.2 , MINDI , ...",andhra_pradesh
9,1.200000,1.10000,1.000000,1343.0,962.0,771.0,23.0,16.0,4.0,1.8,...,7.2,1.200000,1.10000,1.000000,1343.0,962.0,771.0,1522.0,"O/W. PEDDANUYYI - VIZIANAGARAM, A.P",andhra_pradesh
10,1.000000,1.00000,1.000000,347.0,347.0,347.0,3.0,3.0,3.0,0.2,...,7.9,1.000000,1.00000,1.000000,347.0,347.0,347.0,1523.0,"B/W. - NEAR M/S ANDHRA SUGARS LTD. , KOVVUR , ...",andhra_pradesh
11,1.200000,1.20000,1.200000,1904.0,1904.0,1904.0,4.0,4.0,4.0,1.5,...,7.4,1.200000,1.20000,1.200000,1904.0,1904.0,1904.0,1524.0,"O/W. -NEAR PARTAP NAGAR BRIIDGE -KAKINADA , E....",andhra_pradesh
13,1.000000,1.00000,1.000000,1462.0,1462.0,1462.0,10.0,10.0,10.0,5.0,...,7.5,1.000000,1.00000,1.000000,1462.0,1462.0,1462.0,1810.0,"B/W - SRI RAMNAGAR COLONY, SAKKAR NAGAR, BODHA...",andhra_pradesh
14,1.800000,1.80000,1.800000,4010.0,4010.0,4010.0,10.0,10.0,10.0,2.0,...,6.9,1.800000,1.80000,1.800000,4010.0,4010.0,4010.0,1811.0,"B/W. - PRIMARY SCHOOL - RUDRAVELLI (V) , BIBIN...",andhra_pradesh
